In [1]:
%%capture
!pip install -U -q "google-generativeai>=0.7.2"

In [2]:
from google.colab import userdata
import google.generativeai as genai
import pandas as pd
from transformers import AutoTokenizer
import warnings

genai.configure(api_key=userdata.get("GEMINI_KEY"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings("ignore")

# Load data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DATASETS/alpaca_to_translate_glosa.csv")
df['id'] = df['id'].str.replace("_", "", regex=False)
df.head()

,id,instance,class_type,tokens_est_instance,tokens_est_id
0,0ins,Dê três dicas para se manter saudável.,ins,16,3
1,2ins,Descreva a estrutura de um átomo.,ins,15,3
2,3ins,Como podemos reduzir a poluição do ar?,ins,16,3
3,4ins,Descreva uma ocasião em que você teve que tomar uma decisão difícil.,ins,28,3
4,5ins,Identifique o estranho.,ins,8,3


In [ ]:
df.query('id == "10750ins"')

,id,instance,class_type,tokens_est_instance,tokens_est_id
10746,10750ins,"Dê uma alternativa adequada para a frase ""uma mosca na sopa"".",ins,21,4


In [ ]:
# Estimar a quantidade de tokens total por intância
df['total_tokens_est'] = df['tokens_est_instance'] + df['tokens_est_id']
df['acumulado'] = df['total_tokens_est'].cumsum()

# Separar em buckets, para envio em lote para o modelo
# Após um previo teste, identifiquei que o retorno de tokens possui a variação entre 5 e 22 % da quantidade inicial de tokens
# Isso para as instâncias, vou colocar um valore referente a ~45% de espaço para a saída, foi pouco, vamos reduzir ainda mais, para 3000
# Pois espera-se que o chat retorne mais tokens dado as colunas novas e o reescrito, junto ao texto que foi enviado
df['bucket'] = (df['acumulado'] // 3000) + 1
df['bucket'].nunique()

2046

In [ ]:
df.groupby("class_type")['bucket'].count()

,bucket
class_type,
inp,17630
ins,51994
out,48794


In [ ]:
df.bucket.value_counts().sort_values(ascending=False).head() # fazer experimento com o bucket 1,2 (maiores) e 1420, 1683 (menores)

,count
bucket,
2,163
1,161
3,160
4,160
16,157


In [ ]:
df.query("bucket == 1")['total_tokens_est'].sum()

3685

# Transform data in text by bucket

In [ ]:
data_to_text = df[['id', 'instance', 'bucket']].rename(columns={'id':'ID', 'instance':'ORIGINAL'}).copy()
data_to_text.head()

,ID,ORIGINAL,bucket
0,0ins,Dê três dicas para se manter saudável.,1
1,2ins,Descreva a estrutura de um átomo.,1
2,3ins,Como podemos reduzir a poluição do ar?,1
3,4ins,Descreva uma ocasião em que você teve que tomar uma decisão difícil.,1
4,5ins,Identifique o estranho.,1


In [ ]:
data_text = data_to_text.groupby('bucket')[['ID', 'ORIGINAL']].apply(lambda x: x.values.tolist()).tolist()
len(data_text)

2046

In [ ]:
type(data_text[0]), len(data_text[0]), type(data_text[1]), len(data_text[1])

(list, 161, list, 163)

In [ ]:
# Adiciona cabeçalho a todas as sublistas em data_text
for sublist in data_text:
    sublist.insert(0, ['ID', 'ORIGINAL'])

type(data_text[0]), len(data_text[0]), type(data_text[1]), len(data_text[1])

(list, 162, list, 164)

In [ ]:
# Transforma cada sublista em string
for i in range(len(data_text)):
    data_text[i] = str(data_text[i])

type(data_text[0]), len(data_text[0]), type(data_text[1]), len(data_text[1])

(str, 11222, str, 11339)

In [ ]:
# for testing
indices = [1, 2, 1420, 1683]
selected_sublists = [data_text[i] for i in indices if i < len(data_text)]

len(selected_sublists)

4

# Model

In [3]:
model_info = genai.get_model('models/gemini-1.5-flash')
print(f"Informações do modelo, quantidade de tokens para input:{model_info.input_token_limit} \n\
Quantiddade de tokens para output: {model_info.output_token_limit}") #(1048576, 8192)

Informações do modelo, quantidade de tokens para input:1000000 
Quantiddade de tokens para output: 8192


In [4]:
instruction = """Você é um assistente que vai analisar uma base de dados para identificar figuras de linguagem ou expressões idiomáticas.
                 Para cada entrada, você deve:
                 1. Identificar as figuras de linguagem ou expressões idiomáticas na coluna (ORIGINAL).
                 2. Coloque o trecho que represente a figura de linguagem ou expressão na coluna (FIGURATIVO).
                 3. Reescrever o trecho identificado de forma literal (sem a figura de linguagem) em (REESCRITO).
                 4. Substituir o trecho original na frase (ORIGINAL) e coloque em (RESULTADO).
                 5. Retorne somente um json com os campos ID, ORIGINAL, FIGURATIVO, REESCRITO, RESULTADO."""


model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=instruction)

print(f"Quantidade de Instructions Parte 1 (figurativo): {model.count_tokens(instruction)}")

Quantidade de Instructions Parte 1 (figurativo): total_tokens: 304



In [ ]:
print(f"Text1: {model.count_tokens(selected_sublists[0])}")
print(f"Text2: {model.count_tokens(selected_sublists[1])}")
print(f"Text3: {model.count_tokens(selected_sublists[2])}")
print(f"Text4: {model.count_tokens(selected_sublists[3])}")

Text1: total_tokens: 3515

Text2: total_tokens: 3479

Text3: total_tokens: 2684

Text4: total_tokens: 2809



# Identify Figurative Languages

In [ ]:
import os
import time
import logging
from datetime import datetime, timedelta
import google.generativeai as genai
from tqdm import tqdm

logging.basicConfig(filename='/content/drive/MyDrive/Colab Notebooks/DATASETS/TESTE/process_log.txt',
                    level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')


def save_metadata_to_txt(metadata, index, file_path):
    with open(file_path, 'a') as file:
        file.write(f'bucket {index}\n')
        file.write(str(metadata) + '\n')
    logging.info(f"Metadados do bucket {index} salvos com sucesso.")


def save_cleaned_response_to_txt(response_text, file_path):
    cleaned_response_text = response_text.replace("``` json", "")
    cleaned_response_text = response_text.strip().strip("'''").strip('"""')
    cleaned_response_text = response_text.strip().strip("´´´").strip("´´´")
    cleaned_response_text = response_text.strip().strip("```").strip("```")
    with open(file_path, 'a') as file:
        file.write(cleaned_response_text + '\n')
    logging.info("Resposta salva com sucesso no arquivo de texto.")


def instantiate_model():
    instruction = """Você é um assistente que vai analisar uma base de dados para identificar figuras de linguagem ou expressões idiomáticas.
                     Para cada entrada, você deve:
                     1. Identificar as figuras de linguagem ou expressões idiomáticas na coluna (ORIGINAL).
                     2. Coloque o trecho que represente a figura de linguagem ou expressão na coluna (FIGURATIVO).
                     3. Reescrever o trecho identificado de forma literal (sem a figura de linguagem) em (REESCRITO).
                     4. Substituir o trecho original na frase (ORIGINAL) e coloque em (RESULTADO).
                     5. Retorne somente um json com os campos ID, ORIGINAL, FIGURATIVO, REESCRITO, RESULTADO."""

    model = genai.GenerativeModel( model_name="gemini-1.5-flash",
                                   system_instruction=instruction)
    logging.info("Modelo instanciado.")
    return model

def process_data(data_text):
    start_time = datetime.now()
    model = instantiate_model()
    response_file_path = "/content/drive/MyDrive/Colab Notebooks/DATASETS/TESTE/response_data.txt"
    metadata_file_path = "/content/drive/MyDrive/Colab Notebooks/DATASETS/TESTE/response_metadata.txt"

    for i, data in tqdm(enumerate(data_text), total=len(data_text), desc="Processando entradas"):
        try:

            if datetime.now() - start_time > timedelta(hours=1):
                model = instantiate_model()
                start_time = datetime.now()
                logging.info(f"Modelo re-instalado após 1h de execução, continuando do índice {i}.")

            logging.info(f"Iniciando processamento do bucket {i}")

            response = model.generate_content(data)

            save_cleaned_response_to_txt(response.text, response_file_path)

            if response.usage_metadata:
                save_metadata_to_txt(response.usage_metadata, i, metadata_file_path)
            else:
                logging.warning(f"Bucket {i}: Nenhum metadado de uso encontrado.")

        except Exception as e:
            logging.error(f"Ocorreu um erro ao processar o bucket {i}: {e}")

    logging.info("Processamento concluído.")


process_data(selected_sublists)

Processando entradas: 100%|██████████| 4/4 [01:36<00:00, 24.17s/it]


In [ ]:
# para fazer tudo gastarei aproximadamente 12 horas48 minutos

# Translate to glosa.

In [5]:
instruction = """
Atue como um tradutor de LIBRAS (Língua Brasileira de Sinais).
Textos em português serão enviados e você deve converter para a estrutura de GLOSA, respeitando as seguintes regras:
Estrutura de GLOSA mais comum: Objeto + Sujeito + Verbo (OSV) ou Sujeito + Verbo + Objeto (SVO).

Exemplo de few-shot learning:
ID, PT, GLOSA
305, Você sabe onde fica a loja que vende e aluga apartamentos?, VOCÊ SABER ONDE LOJA CASA APARTAMENTO VENDA ALUGUEL ?
75, Você recebeu o salário em dinheiro? Acautele-se e esconda-o no sapato., VOCÊ RECEBER JÁ SÁLARIO DINHEIRO? OLHAR JÁ ESCONDER SAPATO.
4543	Antes eu ganhava pouco, só recebia bolsa de estudo; agora prosperei, ganho bem.	EU ANTES GANHAR POUCO SÓ RECEBER DINHEIRO BOLSA AGORA EU PROSPERAR GANHAR BEM.
4506	No princípio eu não sabia como fazer o dicionário no computador, agora já acostumei.	PRINCÍPIO EU SABER NADA COMO FAZER DICIONÁRIO COMPUTADOR AGORA JÁ ACOSTUMAR.
131	Aqui no Brasil transforma-se a cana em açúcar. Lá na Europa é diferente, transforma-se a beterraba em açúcar.	BRASIL AQUI CANA TRANSFORMAR AÇÚCAR LÁ EUROPA DIFERENTE BETERRABA TRANSFORMAR AÇÚCAR.
3254	Os animais e os bebês precisam tomar leite, é bom para ajudar contra as doenças.	QUALQUER ANIMAL OU BEBÊ PRECISAR TOMAR LEITE BOM AJUDAR CONTRA DOENÇA.
5394	Aqui no Paraná, em julho, sempre há toró. Prefiro viajar para Fortaleza, onde sempre há sol.	PARANÁ JULHO SEMPRE TORÓ EU PREFERIR VIAJAR FORTALEZA LÁ SEMPRE SOL.
305	Você sabe onde fica a loja que vende e aluga apartamentos?	VOCÊ SABER ONDE LOJA CASA APARTAMENTO VENDA ALUGUEL ?
4459	Eu tenho biscoitos salgados e doces, qual você prefere?	EU TER BISCOITO SAL DOCE VOCÊ PREFERIR QUAL?
2937	Você vai comprar carro!? Impossível! Onde irá conseguir dinheiro?	VOCÊ VAI COMPRAR CARRO!? IMPOSSÍVEL! ONDE CONSEGUIR DINHEIRO?
3477	Qual você prefere com o pão: manteiga ou queijo?	PÃO VOCÊ PREFERIR MANTEIGA OU QUEIJO QUAL?
4970	O sabor desse suco parece de tangerina ou laranja. Qual é?	SUCO SABOR PARECER TANGERINA OU LARANJA QUAL?
4024	Ontem fui à padaria comprar pão e levei um susto com o aumento do preço!	ONTEM EU IR PADARIA COMPRAR PÃO EU SUSTO PREÇO AUMENTAR.
4865	É melhor retirar um pouco o que você escreveu, eu não gostei!	VOCÊ ESCREVER EU NÃO-GOSTAR MELHOR VOCÊ RETIRAR POUCO.
4135	O carro entrou na São Paulo-Santos e paguei o valor do pedágio com aumento!	SÃO-PAULO SANTOS CARRO ENTRAR PAGAR PEDÁGIO VALOR AUMENTAR!
4679	No Rio de Janeiro há poucos rabinos, em São Paulo há mais!	RABINO RIO TER POUCO SÃO-PAULO TER MAIS!
283	Ontem, eu estava com enxaqueca, tomei comprimido, a cabeça melhorou e aliviou!	ONTEM EU ENXAQUECA TOMAR-COMPRIMIDO CABEÇA MELHOR ALIVIAR!

Faça o que se pede:
1. Traduza a frase reescrita PT para GLOSA e insira o resultado na coluna GLOSA.
2. Retorne somente um json com os campos  ID, PT, GLOSA.

"""
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    system_instruction=instruction
)

model.count_tokens(instruction)

total_tokens: 1826

In [6]:
data = '''
ID,PT
23870,Reformule esta frase usando um advérbio.
33438,"Analise a seguinte citação: ""A educação é o instrumento mais eficaz que você pode usar para transformar o mundo""."
19819,Identifique o protocolo da Internet que está sendo usado na saída fornecida.
27451,"Dado um conjunto de dados, examine o padrão e forneça um rótulo ou nome descritivo."
13320,Crie uma paisagem sonora baseada na ideia de verão.
46164,Leia as opções de resposta e escolha a correta.
18269,Encontre o título mais adequado para este artigo.
15676,"O protagonista deve tentar encontrar uma saída da sala. Eles podem tentar quebrar a janela, se houver, ou procurar tábuas soltas no piso que levem a um espaço para rastejar ou porão. Outra opção é tentar encontrar uma arma improvisada para dominar o ladrão e recuperar a chave. Eles também podem procurar algo na sala - como um cabide de arame - que possam usar para arrombar a fechadura."
38379,"Um líder deve ter fortes habilidades de tomada de decisão, capacidade de inspirar outras pessoas, boas habilidades de comunicação e capacidade de ter uma visão ampla e correr riscos. Além disso, um líder também deve ser capaz de demonstrar adaptabilidade e autoconsciência."
14528,"No aprendizado de máquina supervisionado, um recurso é um atributo ou característica dos dados usados ​​como entrada em um modelo preditivo. Os recursos podem ser numéricos, categóricos ou baseados em texto e são usados ​​para treinar os algoritmos de aprendizado de máquina e fazer previsões."
38830,"Oi pessoal! Então, recentemente me tornei pai e, deixe-me dizer, é uma jornada louca. Ser pai é comparável a estar em uma montanha-russa que nunca para. Você pode ter um momento de descanso, mas sabe que ela sempre começará a se mover novamente. Mas não são apenas os ciclos intermináveis ​​da paternidade que podem afetar você. É o incômodo. Essa é a pior parte. Você sabe, quando as crianças pensam que podem se safar, é aí que a irritação começa. Então aprendi que às vezes é preciso fazer um acordo. Você sabe, se eles limparem o quarto, eles ganham um biscoito. Ou, se terminarem o dever de casa, poderão assistir a um programa extra. Aquele tipo de coisa. É um equilíbrio delicado, mas é a minha maneira de sobreviver à paternidade. Mas tudo vale a pena no final. Pelo menos é isso que continuo dizendo a mim mesmo. Obrigado, pessoal!"
37082,O silêncio foi tão intenso e alto quanto um trovão.
'''
model.count_tokens(data)

total_tokens: 1527

In [7]:
response = model.generate_content(data)
print(response.text)

```json
[
  {
    "ID": 23870,
    "PT": "Reformule esta frase usando um advérbio.",
    "GLOSA": "FRASE REFORMULAR ADVÉRBIO."
  },
  {
    "ID": 33438,
    "PT": "Analise a seguinte citação: \"A educação é o instrumento mais eficaz que você pode usar para transformar o mundo\".",
    "GLOSA": "CITAÇÃO ANALISAR EDUCAÇÃO INSTRUMENTO EFICAZ VOCÊ PODER USAR TRANSFORMAR MUNDO."
  },
  {
    "ID": 19819,
    "PT": "Identifique o protocolo da Internet que está sendo usado na saída fornecida.",
    "GLOSA": "SAÍDA FORNECER PROTOCOLO INTERNET IDENTIFICAR."
  },
  {
    "ID": 27451,
    "PT": "Dado um conjunto de dados, examine o padrão e forneça um rótulo ou nome descritivo.",
    "GLOSA": "CONJUNTO DADO EXAMINAR PADRÃO RÓTULO NOME DESCRITIVO FORNECER."
  },
  {
    "ID": 13320,
    "PT": "Crie uma paisagem sonora baseada na ideia de verão.",
    "GLOSA": "VERÃO IDEIA PAISAGEM SONORA CRIAR."
  },
  {
    "ID": 46164,
    "PT": "Leia as opções de resposta e escolha a correta.",
    "GLOSA": "OP

In [10]:
response.finish_reason

AttributeError: 'GenerateContentResponse' object has no attribute 'finish_reason'

In [ ]:
#prompt_token_count: 844
#candidates_token_count: 1126
#total_token_count: 1970

In [ ]:
response.usage_metadata

prompt_token_count: 1528
candidates_token_count: 1400
total_token_count: 2928